# XGBOOST ( Extreme Gradient Boosting )

* **Gradinet Boosting Machines'in hız ve tahmin performansını artırmak üzere optimize edilmiş ve bir çok platformda kullanılabilir ve ölçeklenebilir halidir.**

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor  #Random forest için
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
#veri
df = pd.read_csv("beyzbol_veri.csv")
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

### Model & Tahmin

In [6]:
#!pip install xgboost

In [7]:
from xgboost import XGBRegressor

In [8]:
xgb_model = XGBRegressor().fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
RMSE

355.4651481224188

### Model Tuning

* **learning rate : overfiting engellemek için kullanılan bir parametredir.**
* **colsample_bytree : ağaçlardan değişkenlerden alınacak alt küme oranını ifade etmektedir.** 

In [9]:
xgb_model = XGBRegressor().fit(X_train, y_train)

In [17]:
params = {"learning_rate": [0.1, 0.5],
         "colsample_bytree":[0.4 ,1],
         "n_estimator": [500, 2000],
         "max_depth": [1, 5, 10]}

In [18]:
xgb_cv = GridSearchCV(xgb_model, params, cv = 10, n_jobs = -1, verbose = 2).fit(X_train, y_train)

Fitting 10 folds for each of 24 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  92 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 233 out of 240 | elapsed:    7.6s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:    7.7s finished


In [19]:
xgb_cv.best_params_

{'colsample_bytree': 1,
 'learning_rate': 0.5,
 'max_depth': 1,
 'n_estimator': 500}

In [22]:
xgb_tune= XGBRegressor(colsample_tree = 1,
                        learning_rate = 0.5,
                        max_depth = 1,
                        n_estimator = 500).fit(X_train, y_train)

In [23]:
y_pred = xgb_tune.predict(X_test)
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
RMSE

363.76695986463625